# BA305 Final Project Python Notebook (How Do Couples Meet and Stay Together?)

Amelia, John, Hannah, Jacob, Rohit

Boston University

BA305 Business Decision-Making with Data

Professor. Karaca

November 29th, 2023


# 1. Data Upload

In [ ]:

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from numpy.lib.function_base import average
import math
%matplotlib inline
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import *
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
import seaborn as sns
from tqdm import tqdm  # progress slider for "for" loops
from sklearn.preprocessing import LabelEncoder
!pip install keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from google.colab import drive
from google.colab.data_table import DataTable as DT
drive.mount('/content/drive')
datingdf = pd.read_csv('/content/drive/My Drive/Colab Data/HCMST_ver_3.04.csv')
datingdf.info()

In [ ]:
#ORIGINAL DATASET
"""pd.set_option('display.max_columns', None)
datingdf.head()"""

# 2. Data Cleaning


In [ ]:
partnereddf = datingdf.loc[(datingdf['qflag'] == 'partnered')]
partnereddf = partnereddf[["s1", "caseid_new", "respondent_race", "partner_race", "ppage", "q9", "respondent_yrsed", "partner_yrsed", "same_sex_couple", "q12", "pppartyid3","relationship_quality", "parental_approval","w5_broke_up","papreligion", "q7b", "w4_broke_up", "w2_broke_up", "w3_broke_up", "how_long_relationship", "hhinc", "children_in_hh"]]
partnereddf = partnereddf.rename(columns={"s1": "married","ppage": "AgeofResp", "q9": "AgeofPartner", "respondent_yrsed":"EDofResp", "partner_yrsed":"EDofPartner", "q12": "PartnerPoliticalParty", "pppartyid3":"RespPoliticalParty", "ppmarit":"LivingArrangement", "papreligion":"partnerreligion", "q7b":"respondentreligion", "hhinc": "HouseholdIncome", "children_in_hh": "kids"})

partnereddf2 = datingdf.loc[(datingdf['qflag'] == 'partnered')]
partnereddf2 = partnereddf2[["s1", "caseid_new", "respondent_race", "partner_race", "ppage", "q9", "respondent_yrsed", "partner_yrsed", "same_sex_couple", "q12", "pppartyid3","relationship_quality", "parental_approval","w5_broke_up","papreligion", "q7b", "w4_broke_up", "w2_broke_up", "w3_broke_up", "how_long_relationship", "hhinc", "children_in_hh", "how_long_ago_first_cohab", "q32_internet", "how_long_ago_first_met"]]
partnereddf2 = partnereddf2.rename(columns={"q32_internet":"internetmet", "s1": "married","ppage": "AgeofResp", "q9": "AgeofPartner", "respondent_yrsed":"EDofResp", "partner_yrsed":"EDofPartner", "q12": "PartnerPoliticalParty", "pppartyid3":"RespPoliticalParty", "ppmarit":"LivingArrangement", "papreligion":"partnerreligion", "q7b":"respondentreligion", "hhinc": "HouseholdIncome", "children_in_hh": "kids"})

### a. Functions For Data Cleaning

In [ ]:
#Fill in Null Values & Rename Columns
def nullfill(df):
  avglengthrelationship = df['how_long_relationship'].mean()
  df['how_long_relationship'].fillna(avglengthrelationship, inplace=True)
  #df = df.loc[(df["how_long_relationship"] <= 20)]
  df['AgeofPartner'].fillna(df['AgeofResp'], inplace=True)
  df["partner_race"].fillna(df["respondent_race"], inplace=True)
  df["respondent_race"].fillna(df["partner_race"], inplace=True)
  df['EDofPartner'].fillna(df['EDofResp'], inplace=True)
  df['partnerreligion'].fillna(df['respondentreligion'], inplace=True)
  df["agegap"] = abs(df["AgeofResp"] - df["AgeofPartner"])
  df["difrace"] = df["respondent_race"] == df["partner_race"]
  df["haskids"] = (df["kids"] > 0)
  df["difpoliticalparty"] = df["PartnerPoliticalParty"] == df["RespPoliticalParty"]
  return df


#Create Dummy Variables
def DummyVariables(df):
  df = pd.concat([df, pd.get_dummies(df['same_sex_couple'])], axis=1)
  df = pd.concat([df, pd.get_dummies(df['parental_approval'], prefix = "ParentApproval")], axis=1)
  df["educationdifference"] = abs(df["EDofResp"] - df["EDofPartner"])
  df = pd.concat([df, pd.get_dummies(df['relationship_quality'],prefix = "RelationshipQuality")], axis=1)
  df["difreligion"] = df["partnerreligion"] == df["respondentreligion"]
  df['w2_broke_up'].fillna(df['w3_broke_up'], inplace=True)
  df['w2_broke_up'].fillna(df['w4_broke_up'], inplace=True)
  df['w2_broke_up'].fillna(df['w5_broke_up'], inplace=True)
  df['w3_broke_up'].fillna(df['w4_broke_up'], inplace=True)
  df['w4_broke_up'].fillna(df['w5_broke_up'], inplace=True)
  df = pd.concat([df, pd.get_dummies(df['w2_broke_up'], prefix = "1year")], axis=1) #FIXTHISFIX
  df = pd.concat([df, pd.get_dummies(df['w3_broke_up'], prefix = "2year")], axis=1) #FIXTHISFIX
  df = pd.concat([df, pd.get_dummies(df['w4_broke_up'], prefix = "3year")], axis=1) #FIXTHISFIX
  df = pd.concat([df, pd.get_dummies(df['w5_broke_up'], prefix = "4year")], axis=1)
  df = pd.concat([df, pd.get_dummies(df['married'])], axis=1)
  df = df.rename(columns={"yes, i am married": "married"})
  return df


def changebrokeup(df):
    #df['2year_broke up'].fillna(1, inplace=True)
    df['2year_broke up'] = np.where(df['1year_broke up']==1, 1, df['2year_broke up'])
    df['3year_broke up'] = np.where(df['2year_broke up']==1, 1, df['3year_broke up'])
    df['4year_broke up'] = np.where(df['3year_broke up']==1, 1, df['4year_broke up'])
    return df


#CUT THE DATASET TO BE NEW COLUMNS ONLY FOR TESTING
def convert_to_type(df):
  df["difreligion"] = df["difreligion"].astype(int)
  df["difrace"] = df["difrace"].astype(int)
  df["difpoliticalparty"] = df["difpoliticalparty"].astype(int)
  #df["haskids"] = df["haskids"].astype(int)
  return df



#print columns for datasets
def printcol(df):
  for col in df.columns:
    print(col)



### b. Create Datasets

In [ ]:
partnereddf = nullfill(partnereddf)
test1year = DummyVariables(partnereddf)
test1year = changebrokeup(test1year)

#THIS DATASET IS FOR 4 YEAR BROKE UP
testing4dataset = test1year.iloc[:, 19:]
testing4dataset = testing4dataset.drop(['different sex couple', "1year_broke up", "3year_broke up", "2year_broke up", "kids",
                          "1year_partner passed away","3year_partner passed away","2year_partner passed away", "4year_partner deceased","3year_still together", "1year_partner deceased", "3year_partner deceased",
                            "4year_still together",	"1year_still together",	"2year_partner deceased",	"2year_still together", "no, i am not married", "ParentApproval_don't approve or don't know"], axis = 1)
testing4dataset = testing4dataset.rename(columns={"4year_broke up": "Study_Break_Up"})
testing4dataset = convert_to_type(testing4dataset)

#THIS DATASET IS FOR TESTING MORE SIGNIFICANT VARIABLES
partnereddf2 = nullfill(partnereddf2)
test1year2 = DummyVariables(partnereddf2)
test1year2 = changebrokeup(test1year2)
testing4dataset2 = test1year2.iloc[:, 19:]
avgcohab = testing4dataset2['how_long_ago_first_cohab'].mean()
testing4dataset2['how_long_ago_first_cohab'].fillna(avgcohab, inplace=True)
avgmet = testing4dataset2['how_long_ago_first_met'].mean()
testing4dataset2['how_long_ago_first_met'].fillna(avgmet, inplace=True)
testing4dataset2["internetmet"].fillna(0, inplace=True)
testing4dataset2 = testing4dataset2.drop(["educationdifference", "haskids", 'different sex couple', "1year_broke up", "3year_broke up", "2year_broke up", "kids",
                       "1year_partner passed away","3year_partner passed away","2year_partner passed away", "4year_partner deceased","3year_still together", "1year_partner deceased", "3year_partner deceased",
                       "4year_still together", "1year_still together",	"2year_partner deceased",	"2year_still together", "no, i am not married", "ParentApproval_don't approve or don't know"], axis = 1)
testing4dataset2 = testing4dataset2.rename(columns={"4year_broke up": "Study_Break_Up"})
testing4dataset2 = convert_to_type(testing4dataset2) #WON'T RUN UNLESS HAS KIDS IS #'d out
testing4dataset2.info()


In [ ]:
## adding a binned and unbinned version of the df so you can use it for NB and the rest of the applicaitons
## making the dataframe to test the education question I asked down a bunch of kernels
unbinned_df = testing4dataset
for_edu = unbinned_df.copy(deep = False)
unbinned_df.head().to_csv('/content/drive/My Drive/Colab Data/unbinned_df_head.csv')
unbinned_df2 = testing4dataset2

In [ ]:
#TEST TO SEE HOW MANY TRUE BREAK UPS THERE ARE
print(unbinned_df["Study_Break_Up"].value_counts()[1])
print(unbinned_df2["Study_Break_Up"].value_counts()[1])

In [ ]:
print("------------ DATASET VARIABLES --------------")
printcol(unbinned_df)

### c. Export Datasets

In [ ]:
unbinned_df.to_csv('/content/drive/My Drive/Colab Data/unbinned_df.csv')

### d. Binning for Naive Bayes
#### Credit Section Chatgpt4

In [ ]:
## binning this one for NB and will also make a copy to Google drive
## pasting the code for the binning function here so its easier to see what you're doing

## this is becoming 'yes_edu_dif' because the binning of this variable was so hard because education is numeric but there isn't a 1:1 dif between levels
## having it just be a boolean is so much better

## ****** Got this through Chatgpt4

unbinned_df['yes_edu_dif'] = (unbinned_df['educationdifference'] != 0).astype(int)

# defining function to create equal proportioned bins for naive bayes
def binning_for_NB(df, column_name, n_bins):

    # checking if the column even is in the dataframe
    if column_name not in df.columns:
        raise ValueError(f"'{column_name}' not found in dataframe columns.")

    # Sort the dataframe based on the specified column
    # The values within the column name are placed in descending order
    df_sorted = df.sort_values(by = column_name)

    # Calculate the size of each bin (3009 / n_bins)
    bin_size = math.ceil(len(df_sorted) / n_bins)

    # Assign labels to bins based off of the range(n_bins)
    bin_ranges = []
    for i in range(n_bins):
        start_idx = i * bin_size
        end_idx = start_idx + bin_size

        # Get the range for the current bin
        start_value = round(df_sorted.iloc[start_idx][column_name])
        if end_idx < len(df_sorted):
            end_value = round(df_sorted.iloc[end_idx][column_name])
        else:
            end_value = round(df_sorted.iloc[-1][column_name])

        # Ensure the bin ranges do not overlap due to rounding
        if i > 0 and start_value <= bin_ranges[-1][1]:
            start_value = bin_ranges[-1][1]

        # Assign the range to each observation in the bin
        bin_ranges.extend([(start_value, end_value)] * bin_size)

    # The last bin might have fewer observations than bin_size, so adjust the labels accordingly
    bin_ranges = bin_ranges[:len(df_sorted)]

    # Convert bin ranges to string format for labeling
    bin_labels = [f"[{start}, {end})" for start, end in bin_ranges]

    # Add the bin labels to the dataframe
    df_sorted[f"{column_name}_bins"] = bin_labels

    # Create new columns for each bin range
    for bin_range in df_sorted[f"{column_name}_bins"].unique():
        col_name = f"{column_name}_{bin_range}"
        df_sorted[col_name] = (df_sorted[f"{column_name}_bins"] == bin_range).astype(int)

    # Restore the original order of the dataframe
    df = df_sorted.sort_index()

    return df

    ## every single new binned column will have an equal proportion of observations within them with this code for naive bayes (- John)


### e. Making Final Binned Dataframe

In [ ]:
## making a new intermediate dataframe to not fuck with the unbinned one
## calling it "unbinned_df_NB"

## binning length of relationship and removing
unbinned_df_NB = binning_for_NB(unbinned_df, 'how_long_relationship', 8)
unbinned_df_NB = unbinned_df_NB.drop(columns = ["how_long_relationship_bins", "how_long_relationship"])

## binning age gap in relatinship
unbinned_df_NB = binning_for_NB(unbinned_df_NB, 'agegap', 6)
unbinned_df_NB = unbinned_df_NB.drop(columns = ["agegap_bins", "agegap"])

## binning household income
unbinned_df_NB = binning_for_NB(unbinned_df_NB, 'HouseholdIncome', 12)
unbinned_df_NB = unbinned_df_NB.drop(columns = ["HouseholdIncome_bins", "HouseholdIncome"])

## removing the final random leftover columns that shouldnt be in with the binned naive bayes dataframe
binned_NB = unbinned_df_NB.drop(columns = ["educationdifference"])


### f. Showing Binned vs Unbinned Dataframes

In [ ]:
## now we have the binned_NB for the Naive Bayes classification model
binned_NB.head(10)

In [ ]:
## and we have unbinned_df for all other classification models
unbinned_df.head(10)
#unbinned_df.info()

# 3. Model Testing

## a. Naive Bayes Classifier Setup and Run

In [ ]:
#Naive Bayes Bernoulli Classifier

# Import our usual packages...
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import statistics
from pandas.core.frame import DataFrame

# Import some useful packages from scikit-learn
from sklearn import naive_bayes, preprocessing
from sklearn.naive_bayes import MultinomialNB # The Naive Bayes algo
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import *
from sklearn.model_selection import train_test_split

In [ ]:
## Naive Bayes Bernoulli Classifier Results
## Using the binned_NB dataframe for this

def NaiveBayesBer(df, variable):

  df['target'] = df[variable]

  y = df["target"]

  X = df.drop(['target', variable], axis = 1)
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0, stratify = y)
  print('Training set:', X_train.shape, 'Testing set:', X_test.shape)

  BernoulliNBModel = naive_bayes.BernoulliNB()
  BernoulliNBModel.fit(X_train, y_train)

  BernoulliNBModel.class_count_

  y_pred_test = BernoulliNBModel.predict(X_test)
  y_pred_train = BernoulliNBModel.predict(X_train)

#y_pred_train = full_clf.predict(X_train)
#y_pred_test = full_clf.predict(X_test)

  print(confusion_matrix(y_test, y_pred_test))
  print("accuracy for training", accuracy_score(y_train, y_pred_train))
  print("accuracy for testing", accuracy_score(y_test, y_pred_test))


In [ ]:
NaiveBayesBer(binned_NB, "Study_Break_Up")

binned_NB.head()

## this is the only code needed use for this df so theres really no need to drop any of the variables afterwards
## both 'target' and 'study_break_up' are present but the defining function makes it so only one is used for classication and the other isnt looked at
## binned_NB = binned_NB.drop(['target'], axis = 1) - This line was up above but I kinda feel like its not doing anything


## b. PCA

In [ ]:
def heatmap(corr_mat):
  plt.figure(figsize = (14,7))
  sns.heatmap(corr_mat, annot = True, fmt = ".1f", vmin = -1, vmax = 1, cmap = 'RdBu');
 # sns.heatmap(corr_mat, annot = True, fmt = ".1f", vmin = -1, vmax = 1, cmap = 'Reds');
 # sns.heatmap(corr_mat, annot = True, fmt = ".1f", vmin = -1, vmax = 1, cmap = 'Greys');

def scree(df, eig):

  # scree plot of eigenvalues
  # define the x values, i.e., components
  xvals = np.arange(df.shape[1]);
  #figure size
  plt.figure(figsize=(10,5))
  #plot in red color
  plt.plot(xvals, eig, 'ro-', linewidth=1)
  #set ticks to every unit
  plt.xticks(xvals)

  #add labels...
  plt.title('Scree Plot')
  plt.xlabel('Principal Component')
  plt.ylabel('Eigenvalue')

  #add a horizontal line in blue for latent root criterion
  plt.axhline(y=1, color='b', linestyle='--')


def runPCA(df):
  heatmap(corr_mat)

  # Scaling Standardization (0 mean, unit variance) is done via the sklearn.preprocessing scale() function
  pcs = PCA()
  pcs.fit(preprocessing.scale(df))

  # view the importance of principal components
  pcsSummary_df = pd.DataFrame({
              'Proportion of variance': pcs.explained_variance_ratio_,
              'Cumulative proportion': np.cumsum(pcs.explained_variance_ratio_)
              })
  pcsSummary_df.round(3)


  eig=pcs.explained_variance_.round(2)
  print(eig)

  scree(df, eig)

In [ ]:
## using the unbinned_df for this one
## need to remove "educationdifference" from this... or could we see if having a binary classifier is better than a "educationdifference"? like having two dif dataframes
## Ill actually just make a copy of the df and do that... might be cool

column_to_remove = "educationdifference"
if column_to_remove in binned_NB.columns:
    binned_nb.drop(column_to_remove, axis = 1, inplace = True)

corr_mat = unbinned_df.corr()
corr_mat

In [ ]:
#PCA FOR unbinned_df

runPCA(unbinned_df)

## possibly look at combining some of the relationship quality variables like excellent and great

## c. K Nearest Neighbors

In [ ]:
# Separate X (input features/aka predictors) from y (target)


def RunKnearest(df, variable):
  outcome = variable
  predictors = list(df.columns)
  predictors.remove(outcome)

  # Store predictors and target into X and y, respectively
  X = df[predictors]
  y = df[outcome]

  X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.3, random_state=0, stratify=y)
  print('Training set:', X_train.shape, 'Testing set:', X_test.shape)

  # Standardize training and testing features using 'StandardScaler()'
  # (a slightly different method than what we did in the PCA lab)

  # the first line defines the scaling object
  scaler = preprocessing.StandardScaler()

  # the second line specifies which data to use to compute means and variances
  scaler.fit(X_train)
  # important: in this step, you should fit the scaler only to training data,
  # and not the testing data. We assume testing data is never available to us
  # in the training stage

  # the third line scales the data using the means and variances computed in the
  # previous step
  X_train_scaled = scaler.transform(X_train)
  X_test_scaled = scaler.transform(X_test)

  # Note, there is no need to scale the target y variable, since this is what
  # we are trying to predict
  # Run the k-NN model with a random guess about the neighboors, set k=1 for instance
  knn = KNeighborsClassifier(n_neighbors=1)
  # Specify the training features (X_train) and the outcome they lead to (y_train)
  # Important: only use the training data at this step. Do not use test data.
  knn.fit(X_train_scaled, y_train)

  # Now that the model is done, we can use it to predict whether the new customers
  # from the testing data, will accept a loan or not. For this, we feed the
  # testing features X_test into the prediction function.
  y_pred = knn.predict(X_test_scaled)
  print('Accuracy for 1 neighbor:', accuracy_score(y_test, y_pred))
  print()

  results = []
  for k in tqdm(range(1, 51, 1)):
      knn = KNeighborsClassifier(n_neighbors=k).fit(X_train_scaled, y_train)

      # create a dictionary to store the results
      results.append({
          'k': k,
          'accuracy': accuracy_score(y_test, knn.predict(X_test_scaled))
      })

  # convert results to a pandas dataframe for better visualization
  results_df = pd.DataFrame(results)
  #print(results_df)
  print(results_df)

  max_val = results_df['accuracy'].max()
  max_val_idx = results_df['accuracy'].idxmax()

  knn = KNeighborsClassifier(n_neighbors= results_df['k'][max_val_idx])
  knn.fit(X_train_scaled, y_train)

  pred_y_train = knn.predict(X_train_scaled)

  pred_y_test = knn.predict(X_test_scaled)



  print()
  print("K Value", (max_val_idx + 1))
  print(confusion_matrix(y_test, pred_y_test))
  print('Accuracy for test', max,':', accuracy_score(y_test, pred_y_test))


  print()
  print(confusion_matrix(y_train, pred_y_train))
  print('Accuracy for train', max,':', accuracy_score(y_train, pred_y_train))


In [ ]:
#RUN K nearest neighbors for unbinned_df

RunKnearest(unbinned_df, 'Study_Break_Up')

## d. Decision Tree


In [ ]:
# https://github.com/parrt/dtreeviz/blob/master/notebooks/dtreeviz_sklearn_visualisations.ipynb
%%capture
!pip install dtreeviz
import dtreeviz

In [ ]:
# Import the Random Forest (RF) classifier
from sklearn.ensemble import RandomForestClassifier
# Visualize the full tree via "plot_tree" function
from sklearn import tree
print("gi")
def tree_vis(tree):
   # First, re-size the figure via matplotlib(otherwise it will be too small)
  plt.figure(figsize=(15,10))
  # Then call plot_tree() function
  # the 'filled' option generates colors in the nodes
  tree.plot_tree(tree, filled=True);

def plot(a, train_scores, test_scores, x, y):
  fig, ax = plt.subplots()
  ax.set_xlabel("alpha")
  ax.set_ylabel("accuracy")
  plt.xlim(0,0.010)
  plt.ylim(x,y)
  ax.plot(a, train_scores, marker="o", label="train", drawstyle="steps-post")
  ax.plot(a, test_scores, marker="o", label="test", drawstyle="steps-post")
  ax.legend()
  plt.show()


def DTmodel(X, y, aval, xplot, yplot):
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1, stratify=y)

  full_clf = DecisionTreeClassifier(random_state=0)

  full_clf.fit(X_train, y_train)

  print('Number of nodes', full_clf.tree_.node_count)

  # Feed input data to model to predict y output, for training data and test data, separately
  y_pred_train = full_clf.predict(X_train)
  y_pred_test = full_clf.predict(X_test)

  # Confusion matrix of full tree on training and test sets
  print("Confusion Matrix on train data (watch out for overfitting!):")
  print("Accuracy score of reduced tree on train data:", accuracy_score(y_train, y_pred_train)*100, "%")
  print(confusion_matrix(y_train, y_pred_train))

  print("Confusion Matrix of full tree on test data:")
  print("Accuracy score of reduced tree on test data:", accuracy_score(y_test, y_pred_test)*100, "%")
  print(confusion_matrix(y_test, y_pred_test))

  # Retrieve all the 'unique' values of penalty alpha and the impurity score
  # they generate, from the full tree. We will use the cost_complexity_pruning_path() function
  # To retrieve the alpha values, we need to call the option 'ccp_alphas'
  alphas = full_clf.cost_complexity_pruning_path(X_train, y_train).ccp_alphas

  clfs = []

  for i in alphas:
    my_clf = DecisionTreeClassifier(random_state=0, ccp_alpha=i)
    my_clf.fit(X_train, y_train)
    clfs.append(my_clf) #this line adds to the fitted tree to the clfs[] array

    # Let's try to find the best alpha to use...
    # We can plot the tree predictive accuracy as a function of alpha
  train_scores = [accuracy_score(y_train, x.predict(X_train)) for x in clfs]
  test_scores = [accuracy_score(y_test, x.predict(X_test)) for x in clfs]

  plot(alphas, train_scores, test_scores, xplot, yplot)

  final_clf = DecisionTreeClassifier(random_state=0, ccp_alpha=aval)
  final_clf.fit(X_train, y_train)
  # Confusion matrix of "final" tree
  y_pred_final = final_clf.predict(X_test)

  print("Accuracy score of reduced tree on test data :", accuracy_score(y_test, y_pred_final)*100, "%")

  print("Confusion Matrix of final reduced pruned tree on test data:")
  print(confusion_matrix(y_test, y_pred_final))
  #print(y_pred_final)
  # Five-fold cross-validation of the decision tree using cross_val_score function
  # We're going to use the tree stored in 'clf' variable
  accuracy_scores = cross_val_score(final_clf, X_train, y_train, cv=5)
  print('Accuracy scores of each fold: ', [f'{acc:.3f}' for acc in accuracy_scores])
  print('5-fold cross-validation mean accuracy:', round(accuracy_scores.mean(),3))
  print("")
  print("")


  # Extract importance values for each feature (column of X)
  importances1 = final_clf.feature_importances_

  # create a dataframe to store the values and their labels
  df3 = pd.DataFrame({'feature': X_train.columns, 'importance': importances1})

  # sort dataframe by descending order, showing the most important feature top
  df3 = df3.sort_values('importance', ascending = False)

  ##running random forests within this function

  # Run the RF classifier, specifying the number of trees to generate
  # Note, being careful about overfitting is not as critical
  # when using random forest classifiers, given they randomize over
  # the data and the feature columns
  # n_estimators is the total number of different random trees to generate

  rf = RandomForestClassifier(
      n_estimators = 10000,
      random_state = 0,
      criterion = 'gini',
      n_jobs = -1)
  rf.fit(X_train, y_train)

  print("Accuracy Score For Random Forest: ", accuracy_score(y_test, rf.predict(X_test))*100, "%")
  print(confusion_matrix(y_test, rf.predict(X_test)))

  # Extract importance values for each feature (column of X)
  importances = rf.feature_importances_

  # create a dataframe to store the values and their labels
  df2 = pd.DataFrame({'feature': X_train.columns, 'importance': importances})

  # sort dataframe by descending order, showing the most important feature top
  df2 = df2.sort_values('importance', ascending = False)

  # plot the importance of each feature
  ax = df2.plot(kind = 'bar', x = 'feature')

  plt.tight_layout()
  plt.show()

In [ ]:
print("Decision Tree Model For Breaking up During the Study")
Xinput = unbinned_df.drop(columns = ['Study_Break_Up'])
yinput = unbinned_df['Study_Break_Up']
aval = .003
DTmodel(Xinput, yinput, aval, .6, .9)

In [ ]:
print("Decision Tree Model For Breaking up During the Study TEST 2")
Xinput = unbinned_df2.drop(columns = ['Study_Break_Up'])
yinput = unbinned_df2['Study_Break_Up']
aval = .0039
DTmodel(Xinput, yinput, aval, .6, .9)

## e. Testing Same Sex Split

In [ ]:
ss4year = unbinned_df.loc[(unbinned_df['same-sex couple'] == 1)]
hetero4year = unbinned_df.loc[(unbinned_df['same-sex couple'] == 0)]

In [ ]:
print("4 year test Hetero")
NaiveBayesBer(hetero4year, "Study_Break_Up")
hetero4year = hetero4year.drop(['target'], axis = 1)


print("4 year test SS")
NaiveBayesBer(ss4year, "Study_Break_Up")
ss4year = ss4year.drop(['target'], axis = 1)

## f. Neural Network

In [ ]:
def NN2(df):
  y = df['Study_Break_Up']
  X = df.drop(['Study_Break_Up'], axis=1)

  from sklearn.preprocessing import StandardScaler
  PredictorScaler=StandardScaler()
  # Storing the fit object for later reference
  PredictorScalerFit=PredictorScaler.fit(X)

  # Generating the standardized values of X and y
  X=PredictorScalerFit.transform(X)

  # build a model
  model = Sequential()
  model.add(Dense(16, input_shape=(X.shape[1],), activation='relu')) # Add an input shape! (features,)
  model.add(Dense(16, activation='relu'))
  model.add(Dense(16, activation='relu'))
  model.add(Dropout(0.2))
  model.add(Dense(1, activation='sigmoid'))
  model.summary()

  # compile the model
  model.compile(optimizer='Adam',
                loss='binary_crossentropy',
                metrics=['accuracy'])

  # early stopping callback
  # This callback will stop the training when there is no improvement in
  # the validation loss for 10 consecutive epochs.
  es = EarlyStopping(monitor='val_accuracy',
                                    mode='max', # don't minimize the accuracy!
                                    patience=10,
                                    restore_best_weights=True)

  # now we just update our model fit call
  history = model.fit(X,
                      y,
                      callbacks=[es],
                      epochs=80, # you can set this to a big number!
                      batch_size=10,
                      validation_split=0.3,
                      shuffle=True,
                      verbose=1)
  print("Test loss:", score[0])
  print("Test accuracy:", score[1])

def NN(df):

  y = df['Study_Break_Up']
  X = df.drop(['Study_Break_Up'], axis=1)

  from sklearn.preprocessing import StandardScaler
  PredictorScaler=StandardScaler()
  # Storing the fit object for later reference
  PredictorScalerFit=PredictorScaler.fit(X)

  # Generating the standardized values of X and y
  X=PredictorScalerFit.transform(X)

  # Split the data into training and testing set
  from sklearn.model_selection import train_test_split
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify = y)



  # Quick sanity check with the shapes of Training and Testing datasets
  print(X_train.shape)
  print(y_train.shape)
  print(X_test.shape)
  print(y_test.shape)

  model = Sequential()
  model.add(Dense(12, activation='relu'))
  model.add(Dense(8, activation='relu'))
  model.add(Dense(8, activation='relu'))
  model.add(Dense(8, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))

  # compile the keras model
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

  model.fit(X_train, y_train, epochs=50, batch_size=10, validation_data=(X_test, y_test))

  score = model.evaluate(X_test, y_test, verbose=0)
  print("Test loss:", score[0])
  print("Test accuracy:", score[1])



In [ ]:
NN(unbinned_df)

In [ ]:
NN2(unbinned_df)

In [ ]:
NN2(unbinned_df2)

# 4. Data Visualizations

##a. Variables & Broke Up Relationships

In [ ]:
unbinned_df ## using this cause its easier for right now

corr_matrix = unbinned_df.corr()

# Computing a normal correlation coefficient
corr = unbinned_df.corr()["Study_Break_Up"]

# Compute the Pearson correlation coefficient
pearson_corr = unbinned_df.corr(method='pearson')["Study_Break_Up"]

# Compute the Spearman correlation coefficient
spearman_corr = unbinned_df.corr(method = 'spearman')["Study_Break_Up"]

# Compute the Kendall correlation coefficient
kendall_corr = unbinned_df.corr(method = 'kendall')["Study_Break_Up"]

pearson_corr

In [ ]:
# Convert to DataFrame and graph
# Computing a normal correlation coefficient
corr = df.corr()["Income"]



def corr_to_graph(df, type_corr):

  # Convert to DataFrame
  correlation_df = pd.Series(df).to_frame(name = "Correlation")

  # Sort the correlations for better visualization
  correlation_df_sorted = correlation_df.sort_values(by = "Correlation", ascending = True)

  # Create a bar chart with numerical values
  plt.figure(figsize = (10, 8))
  bars = plt.barh(correlation_df_sorted.index, correlation_df_sorted['Correlation'], color = 'lightpink')

  # Add the correlation values at the end of each bar
  for bar in bars:
      width = bar.get_width()
      label_x_pos = bar.get_width() if bar.get_width() < 0 else bar.get_width() + 0.02
      plt.text(label_x_pos, bar.get_y() + bar.get_height()/2, f'{width:.2f}', va = 'center')

  plt.xlabel('Correlation Coefficient')
  plt.title(f'{type_corr} Correlation with Outcome Variable')
  plt.tight_layout()  # Adjust layout for better fit

  # Show the chart with numerical values
  plt.show()

In [ ]:
corr_to_graph(corr, '')

In [ ]:
def corr_to_graph2(df, type_corr):

  # Convert to DataFrame
  correlation_df = pd.Series(df).to_frame(name = "Correlation")

  # Sort the correlations for better visualization
  correlation_df_sorted = correlation_df.sort_values(by = "Correlation", ascending = True)
  correlation_df_sorted = correlation_df_sorted.loc[(abs(correlation_df_sorted['Correlation']) >= .1 )]

  # Create a bar chart with numerical values
  plt.figure(figsize = (10, 8))
  bars = plt.barh(correlation_df_sorted.index, correlation_df_sorted['Correlation'], color = 'lightpink')

  # Add the correlation values at the end of each bar
  for bar in bars:
      width = bar.get_width()
      label_x_pos = bar.get_width() if bar.get_width() < 0 else bar.get_width() + 0.02
      plt.text(label_x_pos, bar.get_y() + bar.get_height()/2, f'{width:.2f}', va = 'center')

  plt.xlabel('Correlation Coefficient')
  plt.title(f'{type_corr} Correlation with Study_Break_Up')
  plt.tight_layout()  # Adjust layout for better fit

  # Show the chart with numerical values
  plt.show()

In [ ]:
def change_break(df):
  df = df.loc[(df['qflag'] == 'partnered')]
  df['w2_broke_up'].fillna(df['w3_broke_up'], inplace=True)
  df['w2_broke_up'].fillna(df['w4_broke_up'], inplace=True)
  df['w2_broke_up'].fillna(df['w5_broke_up'], inplace=True)
  df['w3_broke_up'].fillna(df['w4_broke_up'], inplace=True)
  df['w4_broke_up'].fillna(df['w5_broke_up'], inplace=True)
  df = pd.concat([df, pd.get_dummies(df['w2_broke_up'], prefix = "1year")], axis=1) #FIXTHISFIX
  df = pd.concat([df, pd.get_dummies(df['w3_broke_up'], prefix = "2year")], axis=1) #FIXTHISFIX
  df = pd.concat([df, pd.get_dummies(df['w4_broke_up'], prefix = "3year")], axis=1) #FIXTHISFIX
  df = pd.concat([df, pd.get_dummies(df['w5_broke_up'], prefix = "4year")], axis=1)
  df['2year_broke up'] = np.where(df['1year_broke up']==1, 1, df['2year_broke up'])
  df['3year_broke up'] = np.where(df['2year_broke up']==1, 1, df['3year_broke up'])
  df['4year_broke up'] = np.where(df['3year_broke up']==1, 1, df['4year_broke up'])
  df = df.drop(['2year_broke up', '3year_broke up','1year_broke up', '4year_still together','3year_still together', '2year_still together', '1year_still together'], axis = 1)
  df = df.rename(columns={"4year_broke up": "Study_Break_Up"})
  return df

df = change_break(datingdf)
corrdf = df.corr()["Study_Break_Up"]
corr_to_graph2(corrdf, '')

## b. Other

In [ ]:
# Counting occurrences of the binary variable across the x-axis fields
counts = binned_NB.groupby(['how_long_relationship_[0, 2)', 'how_long_relationship_[2, 6)', 'how_long_relationship_[6, 9)', 'how_long_relationship_[9, 13)', 'how_long_relationship_[13, 18)',
                            'how_long_relationship_[18, 26)', 'how_long_relationship_[26, 39)', 'how_long_relationship_[39, 76)'])['ParentApproval_approve'].sum().reset_index()

# Creating a bar chart using Matplotlib
plt.figure(figsize=(10, 6))
bars = plt.bar(range(len(counts)), counts['ParentApproval_approve'], color='pink')

#x_labels = ['{}_{}'.format(field1, field2, field3, field4, field5, field6, field7, field8) for field1, field2, field3, field4, field5, field6, field7, field8 in zip(counts['how_long_relationship_[0, 2)'],
  #        counts['how_long_relationship_[2, 6)'], counts['how_long_relationship_[6, 9)'], counts['how_long_relationship_[9, 13)'], counts['how_long_relationship_[13, 18)'],
 #         counts['how_long_relationship_[18, 26)'], counts['how_long_relationship_[26, 39)'], counts['how_long_relationship_[39, 76)'])]

x_labels = ['{}{}{}'.format(
        (counts.loc[i, 'how_long_relationship_[0, 2)']),
        (counts.loc[i, 'how_long_relationship_[2, 6)']),
        (counts.loc[i, 'how_long_relationship_[6, 9)']),
        (counts.loc[i, 'how_long_relationship_[9, 13)']),
        (counts.loc[i, 'how_long_relationship_[13, 18)']),
        (counts.loc[i, 'how_long_relationship_[18, 26)']),
        (counts.loc[i, 'how_long_relationship_[26, 39)']),
        (counts.loc[i, 'how_long_relationship_[39, 76)'])
        )
    for i in range(len(counts))]


x_labels2 = ["0 - 2", "2 - 6", "6 - 9", "9 - 13", "13 - 18", "18 - 26", "26 - 39", "39 - 76"]


# Adding x-axis labels
plt.xlabel('Relationship Length in Years')
plt.ylabel('Parental Approval')
plt.title('Bar Chart of Parental Approval Across Relationship Lengths')

# Setting x-axis ticks and labels
#plt.xticks(range(len(counts)), x_labels)

# Rotating x-axis labels for better readability if needed
plt.xticks(range(len(counts)), x_labels2)

# Adding data labels on top of bars
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2, yval, round(yval, 2), va='bottom')

plt.tight_layout()
plt.show()

In [ ]:
Breakup_sum= binned_NB['Study_Break_Up'].sum()
Married_sum= binned_NB['married'].sum()

In [ ]:
data = [['Breakup_sum' , int(Breakup_sum)],
 ['Married_sum' , int(Married_sum)]]


In [ ]:
selected_columns = ['Relationship_Status','Count']  # Columns to be selected for the new DataFrame

# Creating a new DataFrame from selected columns
new_df = pd.DataFrame(data, columns= selected_columns)

print(new_df)

In [ ]:
# Creating the bar chart
plt.figure(figsize=(10, 6))  # Set the figure size
plt.figure(figsize=(8, 6))  # Set the figure size (width, height)


bars = plt.bar(new_df.Relationship_Status, new_df.Count, color='pink')  # Create the bar chart

for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2, yval, yval, ha='center', va='bottom')

plt.xlabel('Relationship Status')  # Label for the x-axis
plt.ylabel('Count')  # Label for the y-axis
plt.title('Distribution of Relationship Status')  # Title for the chart

plt.show()

In [ ]:
x_labels = ['{}{}{}{}{}{}{}'.format(
        (counts.loc[i, 'how_long_relationship_[0, 2)']),
        (counts.loc[i, 'how_long_relationship_[2, 6)']),
        (counts.loc[i, 'how_long_relationship_[6, 9)']),
        (counts.loc[i, 'how_long_relationship_[9, 13)']),
        (counts.loc[i, 'how_long_relationship_[13, 18)']),
        (counts.loc[i, 'how_long_relationship_[18, 26)']),
        (counts.loc[i, 'how_long_relationship_[26, 39)']),
        (counts.loc[i, 'how_long_relationship_[39, 76)'])
        )
    for i in range(len(counts))]

# Logistic Regression

In [ ]:
##unbinned_df

# import required packages
%matplotlib inline
import matplotlib.pylab as plt
import math
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import *

# Define X and y
y = unbinned_df['Study_Break_Up']
X = unbinned_df.drop(columns=['Study_Break_Up'])

# Split data
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size = 0.3, random_state = 0, stratify = y)

# Logistic Regression
lr = LogisticRegression(penalty = "none", solver = 'lbfgs')
lr.fit(X_train, y_train)

# Make predictions on test data
y_pred = lr.predict(X_test)

In [ ]:
# Confusion Matrix
print(confusion_matrix(y_test,y_pred))

# accuracy score
accuracy_score(y_test, y_pred)

In [ ]:
# What if we want to change the default probability cutoff of 0.5?

# Extract regressed probabilities using predict_proba
p_pred = lr.predict_proba(X_test)[:,1]
np.round(p_pred,3)

In [ ]:
# Define new cutoff
cutoff = 0.49
y_pred_new = [1 if i > cutoff else 0 for i in p_pred]
y_pred_new

In [ ]:
# confusion matrix with different cutoff
confusion_matrix(y_test,y_pred_new)

In [ ]:
# accuracy score with different cutoff
accuracy_score(y_test, y_pred_new)

In [ ]:
# Store coefficients
lr_int = lr.intercept_[0]
lr_coef = lr.coef_[0]

# print coefficients
print('intercept ', lr_int)
print(pd.DataFrame({'coeff': lr_coef}, index=X.columns))

In [ ]:
# print exponentiated Experience and Training coefficients
print( math.exp(lr_coef[0]) )
print( math.exp(lr_coef[1]) )

# 5. Archive



Create Bins for Metric Variables and Get Dummies


def CreateBin(df, abinval, lbinval, ebinval, ibinval):
  df['agebin'] = pd.cut(df['agegap'], abinval)
  df = pd.concat([df, pd.get_dummies(df['agebin'], prefix = "Age_Gap_")], axis=1)

  df['how_long_relationship'] = pd.cut(df['how_long_relationship'], lbinval)
  df = pd.concat([df, pd.get_dummies(df['how_long_relationship'], prefix = "Relationship_Length_")], axis=1)


  df['educationdifference'] = pd.cut(df['educationdifference'], ebinval)
  df = pd.concat([df, pd.get_dummies(df['educationdifference'], prefix = "education_difference_")], axis=1)

  df['HouseholdIncome'] = pd.cut(df['HouseholdIncome'], ibinval)
  df = pd.concat([df, pd.get_dummies(df['HouseholdIncome'], prefix = "HouseholdIncome_")], axis=1)

  return df



THIS DATASET IS FOR THE 1 YEAR BROKE UP
 "1year_broke up"
testing1dataset = test1year.iloc[:, 19:]
testing1dataset  = testing1dataset.drop(['different sex couple', "2year_broke up", "3year_broke up", "4year_broke up",
                           "1year_partner passed away","3year_partner passed away","4year_partner deceased","3year_still together",
                            "4year_still together",	"1year_still together",	"2year_partner deceased",	"2year_still together", "no, i am not married", "ParentApproval_don't approve or don't know"], axis = 1)
testing1dataset = convert_to_type(testing1dataset)
"agebin" , "educationdifference",

THIS DATASET IS FOR 2 YEAR BROKE UP
 "2year_broke up"
testing2dataset = test1year.iloc[:, 23:]
testing2dataset  = testing2dataset.drop([ "1year_broke up", "3year_broke up", "4year_broke up", 'different sex couple', "3year_broke up", "4year_broke up",
                            "1year_partner passed away","3year_partner passed away","4year_partner deceased","3year_still together",
                          "4year_still together",	"1year_still together",	"2year_partner deceased",	"2year_still together", "no, i am not married", "ParentApproval_don't approve or don't know"], axis = 1)
testing2dataset = convert_to_type(testing2dataset)
"agebin","educationdifference"




THIS DATASET IS FOR ALL YEAR BROKE UP
testing3dataset = test1year.iloc[:, 19:]
testing3dataset  = testing3dataset.drop(['different sex couple', "1year_broke up", "4year_broke up", "2year_broke up",
                            "1year_partner passed away","3year_partner passed away","4year_partner deceased","3year_still together",
                            "4year_still together",	"1year_still together",	"2year_partner deceased",	"2year_still together", "no, i am not married", "ParentApproval_don't approve or don't know"], axis = 1)
testing3dataset = convert_to_type(testing3dataset)
"agebin","educationdifference"




THIS DATASET IS FOR ALL YEAR BROKE UP
testingalldataset = test1year.iloc[:, 19:]
testingalldataset  = testingalldataset.drop(['different sex couple',
                            "1year_partner passed away","3year_partner passed away","4year_partner deceased","3year_still together",
                            "4year_still together",	"1year_still together",	"2year_partner deceased",	"2year_still together", "no, i am not married", "ParentApproval_don't approve or don't know"], axis = 1)
testingalldataset = convert_to_type(testingalldataset)
"agebin","educationdifference",

OG COLUMN NAMES
for col in datingdf.columns:
    print(col)

print(testing1dataset['1year_broke up'].value_counts()[1])
print(testing2dataset['2year_broke up'].value_counts()[1])
print(testing3dataset['3year_broke up'].value_counts()[1])


printcol(testing1dataset)
print("------------NEW DATASET--------------")
printcol(testing2dataset)
print("------------NEW DATASET--------------")
printcol(testingalldataset)

print("1 year test")
NaiveBayesBer(testing1dataset, "1year_broke up")
testing1dataset = testing1dataset.drop(['target'], axis = 1)

print("2 year test")
NaiveBayesBer(testing2dataset, "2year_broke up")
testing2dataset = testing2dataset.drop(['target'], axis = 1)

print("1 year test Hetero")
NaiveBayesBer(hetero1year, "1year_broke up")
hetero1year = hetero1year.drop(['target'], axis = 1)


print("1 year test SS")
NaiveBayesBer(ss1year, "1year_broke up")
ss1year = ss1year.drop(['target'], axis = 1)

print("2 year test Hetero")
NaiveBayesBer(hetero2year, "2year_broke up")
hetero2year = hetero2year.drop(['target'], axis = 1)


print("2 year test SS")
NaiveBayesBer(ss2year, "2year_broke up")
ss2year = ss2year.drop(['target'], axis = 1)

ss1year = testing1dataset.loc[(testing1dataset['same-sex couple'] == 1)]
hetero1year = testing1dataset.loc[(testing1dataset['same-sex couple'] == 0)]


ss2year = testing2dataset.loc[(testing2dataset['same-sex couple'] == 1)]
hetero2year = testing2dataset.loc[(testing2dataset['same-sex couple'] == 0)]

print("Decision Tree Model For 1 Year Break Up")
Xinput = testing1dataset.drop(columns=['1year_broke up'])
yinput =  testing1dataset['1year_broke up']
aval = .002
DTmodel(Xinput, yinput, aval, .85, .98)

print("Decision Tree Model For 2 Year Break Up")
Xinput = testing2dataset.drop(columns=['2year_broke up'])
yinput =  testing2dataset['2year_broke up']
aval = .0021
DTmodel(Xinput, yinput, aval, .7, .9)

RUN k nearest neighbors for test1dataset

RunKnearest(testing1dataset, '1year_broke up')

RUN k nearest neighbors for test2dataset

RunKnearest(testing2dataset, '2year_broke up')